In [1]:
import nltk
from nltk.stem.porter import *
from torch.nn import *
from torch.optim import *
import numpy as np
import pandas as pd
import torch,torchvision
import random
from tqdm import *
from torch.utils.data import Dataset,DataLoader
stemmer = PorterStemmer()

In [4]:
def tokenize(setence):
    return nltk.word_tokenize(setence)

In [5]:
tokenize('how are you ????')

['how', 'are', 'you', '?', '?', '?', '?']

In [6]:
def stem(word):
    return stemmer.stem(word.lower())

In [7]:
stem('organic')

'organ'

In [16]:
def bag_of_words(tokenized_words,all_words):
    tokenized_words = [stem(w) for w in tokenized_words]
    bag = np.zeros(len(all_words),dtype=np.float32)
    for idx,w in enumerate(all_words):
        if w in tokenized_words:
            bag[idx] = 1.0
    return bag

In [17]:
bag_of_words(['lol','testing','programming','vr'],['lol','testing','programming','vr'])

array([1., 0., 0., 1.], dtype=float32)

In [18]:
data = pd.read_csv('./data.csv')

In [20]:
X = data['text']
y = data['target']

In [21]:
all_words = []
all_data = []
tags = []

In [23]:
for X_batch,y_batch in tqdm(zip(X,y)):
    X_batch = tokenize(X_batch)
    new_X = []
    for X_batch_in_X_batch in X_batch:
        new_X.append(stem(X_batch_in_X_batch))
    all_words.extend(new_X)
    all_data.append((new_X,y_batch))
    tags.append(y_batch)

7613it [00:02, 3097.09it/s]


In [27]:
np.random.shuffle(all_words)
np.random.shuffle(all_data)

In [28]:
all_words = sorted(set(all_words))

In [29]:
tags = sorted(set(tags))

In [30]:
np.random.shuffle(all_words)
np.random.shuffle(all_data)

In [31]:
X = []
y = []

In [32]:
for sent,tag in tqdm(all_data):
    bag = bag_of_words(sent,all_words)
    X.append(bag)
    tag = tags.index(tag)
    y.append(tag)

In [33]:
from sklearn.model_selection import *

In [34]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,shuffle=False)

In [ ]:
def get_loss(model,X,y,criterion):
    